In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

# PySpark MLlib
*Official documentation [here](https://spark.apache.org/docs/latest/mllib-guide.html).*

In [2]:
from pyspark import SparkContext
sc = SparkContext("local[*]", "temp")
print sc.version  # should be >= 1.5.1 for distributed matrices

1.5.1


In [3]:
# needed to convert RDDs into DataFrames
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

## MLlib Data Types (based on RDDs)
### Vector
A mathematical vector. MLlib supports both dense vectors, where every entry is stored, and sparse vectors, where only the nonzero entries are stored to save space. Vectors can be constructed with the mllib.linalg.Vectors package.

In [4]:
import numpy as np
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors, SparseVector

# Create a dense vector (1.0, 0.0, 3.0) from a NumPy array.
dv1 = np.array([1.0, 0.0, 3.0])

# Create a dense vector (1.0, 0.0, 3.0) from a Python list.
dv2 = [1.0, 0.0, 3.0]

# Create a SparseVector (1.0, 0.0, 3.0) by specifying its indices and values corresponding to nonzero entries.
sv1 = Vectors.sparse(3, [0, 2], [1.0, 3.0])

# Create a sparse vector (1.0, 0.0, 3.0) by specifying its nonzero entries.
sv2 = Vectors.sparse(3, [(0, 1.0), (2, 3.0)])

print sv1
print sv2

(3,[0,2],[1.0,3.0])
(3,[0,2],[1.0,3.0])


### LabeledPoint
A labeled data point for supervised learning algorithms such as classification and regression. Includes a feature vector and a label (which is a floating-point value). Located in the mllib.regression package.

In [5]:
from pyspark.mllib.regression import LabeledPoint

point1 = LabeledPoint(1.0, np.array([1.0, 0.0, 3.0]))

point2 = LabeledPoint(-1.0, SparseVector(3, [0, 2], [1.0, 3.0]))

print point1.label, point2.label
print "========"
print point1.features, point2.features

1.0 -1.0
[1.0,0.0,3.0] (3,[0,2],[1.0,3.0])


### Local matrix
*Integer*-typed row and column indices and double-typed values, stored on a single machine. 

In [6]:
from pyspark.mllib.linalg import Matrices

# Create a dense matrix ((1.0, 2.0), (3.0, 4.0), (5.0, 6.0))
dm = Matrices.dense(3, 2, [1, 2, 3, 4, 5, 6])

# Create a sparse matrix ((9.0, 0.0), (0.0, 8.0), (0.0, 6.0))
sm = Matrices.sparse(3, 2, [0, 1, 3], [0, 2, 1], [9, 6, 8])

### Distributed matrix 
*Long*-typed row and column indices and double-typed values, stored distributively in one or more RDDs.  They come in three formats:

- **RowMatrix:** Row-oriented distributed matrix without meaningful row indices, e.g. a collection of feature vectors. It is backed by an RDD of its rows, where each row is a local vector. We assume that the number of columns is not huge for a RowMatrix so that a single local vector can be reasonably communicated to the driver and can also be stored / operated on using a single node. 

In [7]:
from pyspark.mllib.linalg.distributed import RowMatrix

# Create an RDD of vectors.
rows = sc.parallelize([[1.0, 2.0], [2.0, 3.0], [3.0, 4.0]])

# Create a RowMatrix from an RDD of vectors.
rowMatrix = RowMatrix(rows)

# return size
m = rowMatrix.numRows()
n = rowMatrix.numCols() 

print "rows: %d\ncols: %d" % (m, n)

rows: 3
cols: 2


- **IndexedRowMatrix:** Similar to a RowMatrix but with meaningful row indices. It is backed by an RDD of indexed rows, so that each row is represented by its index (long-typed) and a local vector.

In [8]:
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix

indexedRows = sc.parallelize([IndexedRow(0, [1.0, 2.0]),
                              IndexedRow(1, [2.0, 3.0]), 
                              IndexedRow(3, [4.0, 5.0])])

indexedRowMatrix = IndexedRowMatrix(indexedRows)

# return size
m = indexedRowMatrix.numRows()
n = indexedRowMatrix.numCols() 

print "rows: %d\ncols: %d" % (m, n)

rows: 4
cols: 2


- **CoordinateMatrix:** (i.e. a distributed sparse matrix) is (essentially) a list of `(Long, Long, Double)`.

In [9]:
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry

matrixEntries = sc.parallelize([MatrixEntry(0, 0, 1.),
                                MatrixEntry(1, 1, 1.),
                                MatrixEntry(2, 2, 1.)])

coordinateMatrix = CoordinateMatrix(matrixEntries)

# return size
m = coordinateMatrix.numRows()
n = coordinateMatrix.numCols() 

print "rows: %d\ncols: %d" % (m, n)

rows: 3
cols: 3


- **BlockMatrix:** A distributed matrix backed by an RDD of MatrixBlocks, where a MatrixBlock is a tuple of ((Int, Int), Matrix), where the (Int, Int) is the index of the block, and Matrix is the sub-matrix at the given index with size rowsPerBlock x colsPerBlock

In [10]:
from pyspark.mllib.linalg.distributed import BlockMatrix

BlockMatrix = coordinateMatrix.toBlockMatrix()

Note, because the matrix is stored in a distributed way, converting between matrix formats is expensive!

## Machine learning in MLlib

Spark supports a number of machine-learning algorithms.

- Classification and Regression
    - SVM, linear regression
    - SVR, logistic regression
    - Naive Bayes
    - Decision Trees
    - Random Forests and Gradient-Boosted Trees
- Clustering
    - K-means (and streaming K-means)
    - Gaussian Mixture Models
    - Latent Dirichlet Allocation
- Dimensionality Reduction
    - SVD and PCA
- It also has support for lower-level optimization primitives:
    - Stochastic Gradient Descent
    - Low-memory BFGS and L-BFGS

### Parallelized SGD

For linear models like SVM, Linear Regression, and Logistic Regression, the cost function we're trying to optimize is essentially an average over the individual error term from each data point. This is particularly great for parallelization.  For example, in linear regression, recall that the gradient is

$$\begin{align}
\frac{\partial \log(L(\beta))}{\partial \beta} &= \frac{\partial}{\partial \beta} \frac{1}{2}\sum_j \|y_j - X_{j \cdot} \cdot \beta\| \\
&= \frac{1}{2}\sum_j \frac{\partial}{\partial \beta} \|y_j - X_{j \cdot} \cdot \beta\| \\
& = \sum_j y_j - X_{j \cdot} \cdot \beta \\
& \approx \sum_{sample \mbox{ } j} y_j - X_{j \cdot} \cdot \beta
\end{align}$$

The key *mathematical properties* we have used are:

1. the error functions are the sum of error contributions of different training instances
1. linearity of the derivative
1. associativity of addition
1. downsampling giving an unbiased estimator

Since the last sum is over the different training instances and these are stored on different nodes, we can parallelize the computation of the gradient in SGD across multiple nodes.  Of course, we still need to maintain the running weight $\beta$ that has to be present on every node (through a broadcast variable that is updated).  Notice that SVM, Linear Regression, and Logistic Regression all have error functions that are just sums over training instances so SGD can be used for all these algorithms.

Spark's [implementation](http://spark.apache.org/docs/latest/mllib-optimization.html#stochastic-gradient-descent-sgd) uses a tunable minibatch size parameter to sample a percentage of the features RDD. For each iteration, the updated weights are broadcast to the executors, and the update is calculated for each data point and sent back to be aggregated.

Parallelization handles increasing number of sampled data points m quite well since there are no interaction terms and each calculation is independent. Controlling how the algorithm iterates to convergence is also important, and can be done with parameters for the total iterations and step size.

In [11]:
from pyspark.mllib.regression import LinearRegressionWithSGD, LinearRegressionModel
from pyspark.mllib.evaluation import RegressionMetrics
import random

# parameters
TRAINING_ITERATIONS = 10
TRAINING_FRACTION = 0.6

# generate the data
data = sc.parallelize(xrange(1,10001)) \
    .map(lambda x: LabeledPoint(random.random(), [random.random(), random.random(), random.random()]))

# split the training and test sets
splits = data.randomSplit([TRAINING_FRACTION, 1 - TRAINING_FRACTION], seed=42)
training, test = (splits[0].cache(), splits[1])

# train the model
model = LinearRegressionWithSGD.train(training, TRAINING_ITERATIONS)

# get r2 score
predictions = test.map(lambda x: (float(model.predict(x.features)), x.label))
print RegressionMetrics(predictions).r2

-0.251058287609


## Spark ML
Spark ML implements the ideas of transformers, estimators, and pipelines by standardizing APIS across machine learning algorithms. This can streamline more complex workflows.

The core functionality includes:
* DataFrames - built off Spark SQL, can be created either directly or from RDDs as seen above
* Transformers - algorithms that accept a DataFrame as input and return a DataFrame as output
* Estimators - algorithms that accept a DataFrame as input and return a Transformer as output
* Pipelines - chaining together Transformers and Estimators
* Parameters - common API for specifying hyperparameters

For example, a learning algorithm can be implemented as an Estimator which trains on a DataFrame of features and returns a Transformer which can output predictions based on a test DataFrame.

Full documentation can be found [here](http://spark.apache.org/docs/latest/ml-guide.html)

## DataFrames and Spark SQL

Spark SQL is the current effort to provide support for writing SQL queries in Spark. Version 1.6.0 supports Hive, Parquet, and other data sources. [Docs](http://spark.apache.org/docs/latest/sql-programming-guide.html)

The key feature of Spark SQL is the use of DataFrames instead of RDDs. A DataFrame is a distributed collection of data organized into named columns, and operations on DataFrames are first parsed through an optimized execution engine which streamlines and may even reorder the request to optimize execution. The keyword to search here is Catalyst.

Under the hood, operations on DataFrames are boiled down to operations on RDDs, but the RDDs are created by the execution engine, and not directly by the user. It is also possible to convert RDDs to DataFrames and vice versa.

The Spark ML package, unlike MLlib, uses DataFrames as inputs and outputs.

**Question:** What is an example of a "bad" sequence of operations which should be reordered for optimal performance?

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

reviews = [("Prose is well-written, but style is an impediment to learning. Should be called 'Reviewing Spark,' not 'Learning Spark'", 0.0),
            ("Nice Headstart to Spark", 1.0),
            ("Start here: Excellent reference for Spark", 1.0),
            ("Insightful and so Spark-tastic!", 1.0),
            ("Good intro but wordy and lacking details in areas", 0.0),
            ("Best of the Books Currently Available", 1.0),
            ("A good resource for people interested in learning Spark", 1.0),
            ("Great Overview", 1.0)]

test_reviews = [("A decent guided tour of Spark and its major components.", 0.0),
                ("10/10 would buy again", 1.0),
                ("it is simple to follow. well organized. straight ...", 1.0),
                ("Just what you need to get started in Apache Spark.", 1.0),
                ("Very good book for learning Spark", 1.0)]

training = sqlContext.createDataFrame(reviews, ["title", "label"])
test = sqlContext.createDataFrame(test_reviews, ["title", "label"])

# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and logistic regression.
tokenizer = Tokenizer(inputCol="title", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
logreg = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, logreg])

model = pipeline.fit(training)

# Make predictions on test documents
prediction = model.transform(test)
selected = prediction.select("title", "label", "prediction")
for row in selected.collect():
    print(row)

Row(title=u'A decent guided tour of Spark and its major components.', label=0.0, prediction=1.0)
Row(title=u'10/10 would buy again', label=1.0, prediction=1.0)
Row(title=u'it is simple to follow. well organized. straight ...', label=1.0, prediction=1.0)
Row(title=u'Just what you need to get started in Apache Spark.', label=1.0, prediction=1.0)
Row(title=u'Very good book for learning Spark', label=1.0, prediction=1.0)


In [13]:
training.show()

+--------------------+-----+
|               title|label|
+--------------------+-----+
|Prose is well-wri...|  0.0|
|Nice Headstart to...|  1.0|
|Start here: Excel...|  1.0|
|Insightful and so...|  1.0|
|Good intro but wo...|  0.0|
|Best of the Books...|  1.0|
|A good resource f...|  1.0|
|      Great Overview|  1.0|
+--------------------+-----+



In [19]:
prediction.show()

+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|               title|label|               words|            features|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|A decent guided t...|  0.0|[a, decent, guide...|(262144,[97,3543,...|[-4.7714032977492...|[0.00839737493866...|       1.0|
|10/10 would buy a...|  1.0|[10/10, would, bu...|(262144,[67599,75...|[-3.1142959220057...|[0.04252139879146...|       1.0|
|it is simple to f...|  1.0|[it, is, simple, ...|(262144,[3370,337...|[-3.0736180319128...|[0.04420869786925...|       1.0|
|Just what you nee...|  1.0|[just, what, you,...|(262144,[3365,370...|[-2.9512720957303...|[0.04967642318936...|       1.0|
|Very good book fo...|  1.0|[very, good, book...|(262144,[29470,32...|[-4.5818316581120...|[0.01013241332411...|       1.0|
+-------

In [14]:
training.printSchema()

root
 |-- title: string (nullable = true)
 |-- label: double (nullable = true)



In [15]:
training.explain(True)

== Parsed Logical Plan ==
LogicalRDD [title#7,label#8], MapPartitionsRDD[95] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Analyzed Logical Plan ==
title: string, label: double
LogicalRDD [title#7,label#8], MapPartitionsRDD[95] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Optimized Logical Plan ==
LogicalRDD [title#7,label#8], MapPartitionsRDD[95] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Physical Plan ==
Scan PhysicalRDD[title#7,label#8]

Code Generation: true


In [16]:
training.filter(training["label"] == 0).select("title").show()

+--------------------+
|               title|
+--------------------+
|Prose is well-wri...|
|Good intro but wo...|
+--------------------+



In [17]:
# Requires Hive to permanently store tables
training.select("title").registerTempTable('text')  # This is NOT the same as a temp table in SQL proper
df = sqlContext.sql("select title, count(*) as total from text group by title having length(title) > 15")
df.show()

+--------------------+-----+
|               title|total|
+--------------------+-----+
|Insightful and so...|    1|
|Good intro but wo...|    1|
|A good resource f...|    1|
|Prose is well-wri...|    1|
|Start here: Excel...|    1|
|Best of the Books...|    1|
|Nice Headstart to...|    1|
+--------------------+-----+



In [18]:
df.explain(True)

== Parsed Logical Plan ==
'Filter ('length('title) > 15)
 'Aggregate ['title], [unresolvedalias('title),unresolvedalias(count(1) AS total#21L)]
  'UnresolvedRelation [text], None

== Analyzed Logical Plan ==
title: string, total: bigint
Filter (length(title#7) > 15)
 Aggregate [title#7], [title#7,count(1) AS total#21L]
  Subquery text
   Project [title#7]
    LogicalRDD [title#7,label#8], MapPartitionsRDD[95] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Optimized Logical Plan ==
Filter (length(title#7) > 15)
 Aggregate [title#7], [title#7,count(1) AS total#21L]
  Project [title#7]
   LogicalRDD [title#7,label#8], MapPartitionsRDD[95] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Physical Plan ==
Filter (length(title#7) > 15)
 TungstenAggregate(key=[title#7], functions=[(count(1),mode=Final,isDistinct=false)], output=[title#7,total#21L])
  TungstenExchange hashpartitioning(title#7)
   TungstenAggregate(key=[title#7], functions=[(count(1),mode=Par

### Tungsten

* Memory management and GC (better than the JVM)
* Cache-aware computation
* Codegen (compile queries into Java bytecode)

Cache-aware computation example:
Case 1: pointer -> key, value
Case 2: ke, pointer -> key, value
Where to find keys for sort purposes?

[More](https://databricks.com/blog/2015/04/28/project-tungsten-bringing-spark-closer-to-bare-metal.html)

## DataFrame performance and tuning

See [here](http://spark.apache.org/docs/latest/sql-programming-guide.html#performance-tuning) for details.

### Example algorithm: Word2Vec

In [20]:
import os
def localpath(path):
    return 'file://' + str(os.path.abspath(os.path.curdir)) + '/' + path

In [21]:
from pyspark.ml.feature import Word2Vec

# text = sc.parallelize(reviews + test_reviews).map(lambda (line, score): (line.split(" "), score)).toDF(['text', 'score'])
gutenberg = sc.textFile(localpath('small_data/gutenberg/')).map(lambda line: (line.split(" "), 1)).toDF(['text', 'score'])
w2v = Word2Vec(inputCol="text", outputCol="vectors")
model = w2v.fit(gutenberg)
result = model.transform(gutenberg)

In [22]:
vectors = model.getVectors().rdd.map(lambda x: (x.word, x.vector))
#print model.findSynonyms('woman', 10).rdd.take(10)

In [30]:
king_vec = vectors.lookup('king')[0]
queen_vec = vectors.lookup('queen')[0]
man_vec = vectors.lookup('man')[0]
woman_vec = vectors.lookup('woman')[0]
fun_vec = vectors.lookup('fun')[0]

In [31]:
print fun_vec

[-0.00365848164074,-0.0065196454525,-0.0804284065962,0.152335777879,0.0326004549861,-0.019918911159,-0.0436401516199,0.0555587783456,-0.0210002548993,-0.0436907336116,0.170062333345,0.0474225617945,0.0547049194574,0.0117957973853,-0.0102078747004,0.0845418348908,0.00444008409977,-0.00810476671904,-0.0137997018173,-0.00498426239938,0.090994335711,-0.0399200767279,-0.0091584995389,-0.0305552836508,0.0126363532618,0.0905004069209,-0.000924758322071,0.067603982985,0.061579503119,-0.120177254081,-0.0262913368642,-0.00785322673619,0.0950697362423,-0.0419631488621,-0.0350100472569,0.076530598104,-0.0146682085469,0.00591638032347,0.192819431424,0.0727502256632,0.0341582782567,-0.0685464963317,-0.0473059006035,0.0641125813127,-0.0584225617349,0.0779399499297,-0.0152564421296,0.0353797115386,0.0428476147354,-0.0386568158865,-0.0168507024646,0.0676571428776,0.00822687707841,0.0100878244266,-0.0634497925639,0.0724031105638,-0.0357816703618,0.0267206374556,0.0137565480545,0.0707818940282,0.00010806

In [24]:
print king_vec

[0.0937515497208,0.0183246023953,0.000950152869336,0.0185837522149,0.0713655352592,-0.0063332375139,-0.0454377606511,0.0473997183144,-0.0285515654832,-0.055686134845,0.127984464169,-0.0252038557082,0.0617561638355,0.000601839739829,-0.0069263051264,0.118331976235,0.0913769304752,0.0498140752316,0.0473664104939,0.0187189653516,0.17080681026,-0.0446416139603,-0.0121530350298,0.0164760164917,0.0329034179449,0.0415324233472,0.0440274849534,0.10448115319,0.115042738616,-0.111048586667,-0.0205034073442,0.0550046488643,0.00128966965713,-0.091121353209,-0.035124886781,0.111250318587,-0.0404968075454,-0.0558430813253,0.0932889133692,-0.0199452191591,-0.0024047892075,-0.0284786876291,-0.0481160841882,0.0962753891945,-0.0534815043211,0.106240496039,0.0274849459529,-0.0146097261459,-0.0200682729483,-0.0649460628629,-0.0845576226711,-0.0514568053186,0.0300664678216,0.0407979972661,-0.104042924941,0.0585550256073,0.0601285584271,-0.0118421986699,-0.0593617074192,0.109998866916,0.11498131603,0.068977

In [37]:
print queen_vec.squared_distance(king_vec)
print queen_vec.squared_distance(woman_vec)
print queen_vec.squared_distance(man_vec)
print queen_vec.squared_distance(king_vec + man_vec - woman_vec)
print queen_vec.squared_distance(king_vec - man_vec + woman_vec)
print fun_vec.squared_distance(man_vec + king_vec)
print fun_vec.squared_distance(woman_vec )

0.229802003101
1.60639476246
2.2375540988
2.65991495515
2.11733516905
1.63887484289
1.0481737307


### Exercise: Use SVM to predict colon cancer from gene expressions
You can start getting a feel for the MLlib operations by following the [Spark docs example](https://spark.apache.org/docs/1.3.0/mllib-linear-methods.html#linear-support-vector-machines-svms) on this dataset.

#### About the data format: LibSVM
MLlib conveniently provides a data loading method, `MLUtils.loadLibSVMFile()`, for the LibSVM format for which many other languages (R, Matlab, etc.) also have loading methods.  
A dataset of *n* features will have one row per datum, with the label and values of each feature organized as follows:
>{label} 1:{value} 2:{value} ... n:{value}

Take these two datapoints with six features and labels of -1 and 1 respectively as an example:
>-1.000000  1:2.080750 2:1.099070 3:0.927763 4:1.029080 5:-0.130763 6:1.265460  
1.000000  1:1.109460 2:0.786453 3:0.445560 4:-0.146323 5:-0.996316 6:0.555759 

#### About the colon-cancer dataset
This dataset was introduced in the 1999 paper "Broad patterns of gene expression revealed by clustering analysis of tumor and normal colon tissues probed by oligonucleotide arrays." (Available on PNAS)

Here's the abstract of the paper:  
> *Oligonucleotide arrays can provide a broad picture of the state of the cell, by monitoring the expression level of thousands of genes at the same time. It is of interest to develop techniques for extracting useful information from the resulting data sets. Here we report the application of a two-way clustering method for analyzing a data set consisting of the expression patterns of different cell types. Gene expression in 40 tumor and 22 normal colon tissue samples was analyzed with an Affymetrix oligonucleotide array complementary to more than 6,500 human genes. An efficient two-way clustering algorithm was applied to both the genes and the tissues, revealing broad coherent patterns that suggest a high degree of organization underlying gene expression in these tissues. Coregulated families of genes clustered together, as demonstrated for the ribosomal proteins. Clustering also separated cancerous from noncancerous tissue and cell lines from in vivo tissues on the basis of subtle distributed patterns of genes even when expression of individual genes varied only slightly between the tissues. Two-way clustering thus may be of use both in classifying genes into functional groups and in classifying tissues based on gene expression.*

There are 2000 features, 62 data points (40 tumor (label=0), 22 normal (label=1)), and 2 classes (labels) for the colon cancer dataset. 

#### Exit Tickets
1. When would you use `org.apache.spark.mllib.linalg.Vector` versus `breeze.linalg.DenseVector`?
1. Why can SVM, Linear Regression, and Logistic Regression be parallelized?  How would you parallelize KMeans?


In [ ]:
sc.stop()

*Copyright &copy; 2015 The Data Incubator.  All rights reserved.*